In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image
from mpl_toolkits.axes_grid1 import ImageGrid

import os, colour, cv2, math, itertools

In [ ]:
def build_directory(directory):
    root = directory[0:2] + "//"
    directory = directory.replace(root, "", 1)
    directory_array = directory.split("/")[1:]
    for i in range(1, len(directory_array) + 1):
        try:
            os.mkdir(root + "/".join(directory_array[0:i]))
        except FileExistsError:
            pass
    
def file_index(files_dir, file):
    try:
        files = os.listdir(files_dir)
        index = files.index(file)
        return index
    except:
        return -1

def norm_array(arr, arr_min, arr_max):
    return (arr - arr_min)/(arr_max - arr_min)

def std_array(arr, arr_avg, arr_std):
    return (arr - arr_avg) / arr_std

def load_image(file_dir) :
    img = Image.open(file_dir)
    img.load()
    data = np.asarray(img, dtype=np.uint8)
    return data

In [ ]:
class Gradient:
    def __init__(self, cmap):
        self.cmap = plt.get_cmap(cmap)
    
    def perc_to_rgb(self, perc, nan=(255, 255, 255)):
        if np.isnan(perc):
            return nan
        else:
            color = self.cmap(int(perc*255))
            return (np.array(color[:-1])*255).round()

In [6]:
base_dir = "C:/Users/Louis Lapp/Documents/Research/Arctic Sea Ice/topographical-sea-ice-forecasting"
image_dir = f"{base_dir}/images_std/"
collage_dir = f"{base_dir}/collages/"
video_dir = f"{base_dir}/videos/"

In [3]:
data = np.load("data/whole_data.npy")

In [4]:
var_names = ["surface_pressure", "wind_velocity", "specific_humidity", "air_temperature", "shortwave_radiation", "longwave_radiation", "rain_rate", "snowfall_rate", "sea_surface_temperature", "sea_ice_concentration"]
var_gradients = ["seismic", "Greys", "Oranges", "coolwarm", "Purples", "Reds", "YlGn", "BuPu", "YlOrRd", "Blues"]
date_range = np.arange(np.datetime64("1979-01"), np.datetime64("2021-07"))

var_arrays = []
for i in range(len(var_names)):
    var_arrays.append(data[:, :, :, i])
    
var_arrays = np.array(var_arrays)

In [115]:
norm_mode = "std"
for var_name, var_gradient, array in zip(var_names, var_gradients, var_arrays):
    if (norm_mode=="norm"):
        arr_min, arr_max = np.nanmin(array), np.nanmax(array)
    elif (norm_mode=="std"):
        arr_avg, arr_std = np.nanmean(array), np.nanstd(array)
    gradient = Gradient(var_gradient)
    save_dir = f"{image_dir}{var_name}/"
    build_directory(save_dir)
        
    for date, month_array in zip(date_range, array):
        print(f"Processing: {var_name} ({date})          ", end="\r")
        try:
            if (file_index(save_dir, f"{date}.png") != -1):
                raise FileExistsError
            shape = month_array.shape + (3, )
            
            if (norm_mode=="norm"):
                month_array = norm_array(month_array.flatten(), arr_min, arr_max)
            elif (norm_mode=="std"):
                month_array = norm_array(std_array(month_array.flatten(), arr_avg, arr_std).clip(-3, 3), -3, 3)
                
            rgb_month_array = []
            for pixel in month_array:
                rgb_month_array.append(gradient.perc_to_rgb(pixel))
            rgb_month_array = np.array(rgb_month_array, dtype=np.uint8).reshape(shape)

            Image.fromarray(rgb_month_array, "RGB").save(f"{save_dir}{date}.png")
        except FileExistsError:
            pass

In [ ]:
build_directory(collage_dir)

for date in np.array(date_range, dtype=str):
    print("Processing:", date, end="\r")
    image_paths = [f"{image_dir}{var}/{date}.png" for var in var_names]
    images = [load_image(image_path) for image_path in image_paths]
    
    plt.style.use('dark_background')
    plt.rcParams["axes.titley"] = 0
    
    fig = plt.figure(figsize=(16, 12))    
    grid = ImageGrid(fig, 111,
                 nrows_ncols=(2, 5),
                 axes_pad=0
                 )
    for ax, im, name in zip(grid, images, var_names):
        ax.imshow(im)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.title.set_text(name)
        
    fig.suptitle(date, x=0.1455, y=0.82)
    fig.savefig(f"{collage_dir}{date}.png", bbox_inches="tight", dpi=100)
    plt.close()

In [7]:
video_name = "data_video.mp4"
images = [img for img in os.listdir(collage_dir) if img.endswith(".png")]
frame = cv2.imread(os.path.join(collage_dir, images[0]))
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, 0, 2, (width,height), 0)
for i, image in enumerate(images):
    print("Processing:", i, end="\r")
    video.write(cv2.imread(os.path.join(collage_dir, image)))
cv2.destroyAllWindows()
video.release()